# Libraries

In [ ]:
import pickle as pkl
import pandas as pd

In [ ]:
data_strategy = "ml1m"  # change depending on which dataset we want to examine

# Analysis

## LKPY

In [ ]:
algo_versions = {
    "UserKNN": [
        {"min_nbrs": 1, "min_sim": 0},
        {"min_nbrs": 2, "min_sim": 0},
        # {'min_nbrs':5, 'min_sim':0},
        {"min_nbrs": 10, "min_sim": 0},
        {"min_nbrs": 1, "min_sim": -1},
        {"min_nbrs": 2, "min_sim": -1},
        # {'min_nbrs':5, 'min_sim':-1},
        {"min_nbrs": 10, "min_sim": -1},
    ],
}

In [ ]:
algo_name = "UserKNN"
versions = algo_versions[algo_name]

In [ ]:
file_location = "experimental_results/" + algo_name + "/"

In [ ]:
results = []
for args in versions:
    file = open(file_location + data_strategy + "_" + str(args) + ".pkl", "rb")
    result = pkl.load(file)
    results.append(result)

In [ ]:
stringed_versions = [str(args) for args in versions]

In [ ]:
# Initialize empty lists for the two halves
min_nbrs = []
min_sim = []

# Split the strings and populate the lists
for s in stringed_versions:
    parts = s.split(", ")

    min_nbrs.append(parts[0].split(" ")[-1])
    min_sim.append(parts[1].split(" ")[-1].split("}")[0])

In [ ]:
over_common = ["False"]

In [ ]:
index = pd.MultiIndex.from_product(
    [min_sim, min_nbrs, over_common],
    names=["MinimumSimilarity", "MinimumNeighbours", "OverCommon"],
).drop_duplicates()

In [ ]:
results_lkpy = pd.DataFrame(results, index=index)

## Cornac

In [ ]:
algo_versions = {
    "CornacUserKNN": [{"center": True}],
}
algo_name = "CornacUserKNN"

In [ ]:
versions = algo_versions[algo_name]

In [ ]:
file_location = "experimental_results/" + algo_name + "/"

In [ ]:
results = []
for args in versions:
    file = open(file_location + data_strategy + "_" + str(args) + ".pkl", "rb")
    result = pkl.load(file)
    results.append(result)

In [ ]:
stringed_versions = [str(args) for args in versions]

In [ ]:
# Initialize empty lists for the two halves
min_nbrs = []
min_sim = []
# Split the strings and populate the lists
for s in stringed_versions:
    parts = s.split(": ")
    min_nbrs.append(parts[-1].split("}")[0])
    min_sim.append("-1")

In [ ]:
over_common = ["True"]

In [ ]:
index = pd.MultiIndex.from_product(
    [min_sim, min_nbrs, over_common],
    names=["MinimumSimilarity", "MinimumNeighbours", "OverCommon"],
).drop_duplicates()

In [ ]:
results_cornac = pd.DataFrame(results, index=index)

In [ ]:
results_cornac.index = results_cornac.index.set_levels(
    results_cornac.index.levels[1].str.replace("True", "1"), level=1
)

In [ ]:
results_cornac

# Merge

In [ ]:
user_knn_metrics = (
    pd.concat([results_lkpy, results_cornac])
    .reset_index()
    .sort_values(["MinimumSimilarity", "OverCommon", "MinimumNeighbours"])
    .set_index(["MinimumSimilarity", "OverCommon", "MinimumNeighbours"])
)

In [ ]:
metrics_order = ["pop_corr", "ARP", "ave_PL", "ACLT", "AggDiv", "RMSE", "NDCG"]
user_knn_metrics = user_knn_metrics[metrics_order]

In [ ]:
user_knn_metrics = user_knn_metrics.rename(
    columns={"pop_corr": "PopCorr", "ave_PL": "PL", "ACLT": "APLT", "NDCG": "NDCG@10"}
).reindex(["1", "2", "5", "10"], level=2)

In [ ]:
print(user_knn_metrics.drop("APLT", axis=1).round(3).to_latex())

In [ ]:
import pickle

with open("metrics_combined/ml1m_all_user_knn.pkl", "wb") as f:
    pickle.dump(user_knn_metrics.drop("APLT", axis=1).round(3), f)